In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [13]:
base_dir = 'img/img/'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5995 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [16]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [19]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(4, activation='softmax', kernel_regularizer=l2(0.01))  # L2 정규화 추가
])

In [20]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# 학습률 감소 콜백 설정
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10,
    callbacks=[reduce_lr]
)

Epoch 1/10
187/187 [==============================] - 454s 2s/step - loss: 1.0834 - accuracy: 0.5606 - val_loss: 1.4555 - val_accuracy: 0.2508 - lr: 0.0010
Epoch 2/10
187/187 [==============================] - 440s 2s/step - loss: 0.7295 - accuracy: 0.7342 - val_loss: 1.4495 - val_accuracy: 0.2483 - lr: 0.0010
Epoch 3/10
187/187 [==============================] - 437s 2s/step - loss: 0.6403 - accuracy: 0.7667 - val_loss: 1.4791 - val_accuracy: 0.2525 - lr: 0.0010
Epoch 4/10
187/187 [==============================] - 424s 2s/step - loss: 0.5737 - accuracy: 0.7905 - val_loss: 1.4307 - val_accuracy: 0.2635 - lr: 0.0010
Epoch 5/10
187/187 [==============================] - 425s 2s/step - loss: 0.5214 - accuracy: 0.8123 - val_loss: 1.5249 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 6/10
187/187 [==============================] - 424s 2s/step - loss: 0.4918 - accuracy: 0.8202 - val_loss: 4677.9229 - val_accuracy: 0.2340 - lr: 0.0010
Epoch 7/10
187/187 [==============================] - 425s 2s

In [22]:
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

38/38 [==============================] - 40s 1s/step - loss: 1.0194 - accuracy: 0.6042
Validation Loss: 1.019365668296814
Validation Accuracy: 0.6041666865348816
